# 04 - Emulating hydrological models

## Using Ravenpy to emulate an existing hydrological model

In this notebook, we will demonstrate the versatility of the Raven modelling framework to emulate one of eight hydrological models that are currently supported. We will walk through the different configuration parameters required to build the model and simulate streamflow on a catchments. We will also show how to import files from a pre-configured Raven configuration that users can inport into Ravenpy instead of using one of the default emulators.

## A note on datasets

There are numerous ways to run a Raven model and to pass its required input data. For this introduction to RavenPy, we will use our ERA5 data we generated in the previous notebook and we will configure the Raven model instance on the fly! In the next tutorials, we will see how users can import and use their own datasets to make the entire process flexible and tailored to the user needs.

## Using templated model emulators
The first thing we need to run the raven model is... a Raven model! Raven is not a model per se, but a modelling framework that can be used to build hydrological models from their underlying components. For now, PAVICS-Hydro allows building a set of pre-determined models. The Python wrapper offers at present eight model emulators: GR4J-CN, HMETS, MOHYSE, HBV-EC, Canadian Shield, HYPR, Sacramento and Blended. For each of these, templated configuration files are available to facilitate launching the model with options passed by Python at run-time.

In the next cell, we are going to import the possible models, and later, we will configure and run the GR4J-CN model. Please see the documentation for more details on the mandatory vs optional parameters, and what they represent. A small glimpse is provided here.

In [1]:
# Import the list of possible model templates.
from ravenpy.new_config.emulators import (
    GR4JCN,
    HBVEC,
    HMETS,
    HYPR,
    SACSMA,
    Blended,
    CanadianShield,
    Mohyse,
)

In [2]:
import datetime as dt

# Import other required packages:
import tempfile
from pathlib import Path

import xarray as xr

from ravenpy.new_config import commands as rc
from ravenpy.utilities.testdata import get_file

In this next step, we will define the hydrological response unit (HRU). For lumped models, there is only one unit so the following structure should be good. However, for distributed modelling, there will be more than one HRU, so we would use another tool to help us build the HRUs in that case. The HRU provides information on the area, elevation, and location of the catchment.

For now, let's provide the basin properties such that Raven can run. These are the minimal values that must always be provided, but some models might require other inputs. Please see the Raven documentation for more information.

In [3]:
# Define the hydrological response unit. We can use the information from the tutorial notebook #02! Here we are using
# arbitrary data for a test catchment.
hru = {}
hru = dict(
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    hru_type="land",
)

The next required inputs are the start and end dates for the simulation. The `start_date` and `end_date` arguments indicate when a simulation should start and end. As long as the forcing data covers the simulation period, it should work. If these parameters are not defined, then start and end dates default to the start and end of the driving data.

To keep things simple, we will use a short 5-year period. Note that the dates are python datetime.datetime objects.

In [4]:
start_date = dt.datetime(1985, 1, 1)
end_date = dt.datetime(1990, 1, 1)

We are now ready to build our first Raven-based hydrological model. the model will be the GR4JCN model. The following code block will show and describe every step. However, more control options are available for users. Please see the documentation for a more detailed explanation on model options.

In [5]:
# Import required packages. We already imported the GR4JCN emulator in the first cell, but let's keep it here for
# completeness.
from ravenpy.new_config.emulators import GR4JCN

# Alternative variable names are useful for allowing Raven to read variables from NetCDF files even if the variable
# names are not those that are expected by Raven. For example, our ERA5-dernived temperature variables are named
# "tmax" and "tmin", whereas Raven expects "TEMP_MAX" and "TEMP_MIN". Therefore, instead of forcing users to rename
# their variables, we provide a mechanism to tell Raven which variables in the NetCDF files correspond to which
# meteorological variable.
alt_names = {
    "TEMP_MIN": "tmin",
    "TEMP_MAX": "tmax",
    "PRECIP": "pr",
}

# Here is where we build the raven configuration. We will first configure the model in memory based on the user
# preferences, and then we will write the Raven configuration files to disk such that Raven can read them and
# execute a simulation. In this case, we are building a GR4JCN model, but you could change this to any of the
# eight models that are preconfigured for direct emulation in Ravenpy.

data_kwds = {
    "ALL": {
        "elevation": hru["elevation"],
        "latitude": hru["latitude"],
        "longitude": hru["longitude"],
    }
}

# Get the weather data. It can either be to a data file that was already in the same folder/workspace as this
# notebook, as we generated in the previous notebook, or it could be a path to a file stored elsewhere.

# Example for using the data we just generated in Notebook 03:
"""
ERA5_tmax = "ERA5_tmax.nc"
ERA5_tmin = "ERA5_tmin.nc"
ERA5_pr = "ERA5_pr.nc"
"""

# In our case, we will prefer to link to existing, pre-computed and locally stored files to keep things tidy:
ERA5_tmax = get_file("notebook_inputs/ERA5_tmax.nc")
ERA5_tmin = get_file("notebook_inputs/ERA5_tmin.nc")
ERA5_pr = get_file("notebook_inputs/ERA5_pr.nc")


m = GR4JCN(
    # Raven requires parameters for the GR4JCN model. For now, we provide default values, but in a later notebook
    # we will show how to calibrate and find new parameters for our model.
    params=[0.529, -3.396, 407.29, 1.072, 16.9, 0.947],
    # Here we will prepare the weather gauge data to be fed to the model. The data could also come from other
    # sources such as the ERA5 reanalysis product. There are 2 ways to do this. We will show one way here, and
    # then show the alternative method in the following cell.
    #
    # METHOD 1: If you have one netcdf file of data per meteorological variable (such as what is generated in the
    #    03_Extracting_forcing_data.ipynb notebook), you can add them successively as follows. Note that we are
    #    creating a gauge for each variable. Raven will use the data from the three sources to provide forcing
    #    data to the simulation.
    #
    #   You can add the files you need! As long as there are timestamps associated with each value in the netcdf
    #   files, and the other required information (data_type, alt_names, other required information), the code
    #   will accept them and use what it needs. As per the Raven model itself, the gauge station elevation, latitude
    #   and longitude are required to let the model run. For lumped models such as the ones we are currently
    #   emulating, there is only one "station" that corresponds to the basin-averaged weahter. In this case, we
    #   set the station elevation to that of the mean catchment elevation to remove any adiabatic gradient
    #   modifications to the data. We also provide the catchment centroid latitude and longitude which we take
    #   from the only HRU defining the entire catchment. For multi-gauge basins and semi-distributed models, the
    #   latitude and longitude must be correctly identified for each station.
    Gauge=[
        rc.Gauge.from_nc(
            ERA5_tmax,  # This is a path to the file of ERA5-derived maximum daily temperature.
            data_type=[
                "TEMP_MAX"
            ],  # Raven expects maximum temperature to be identified as "TEMP_MAX", so we indicate that this variable is maximum temperature.
            alt_names=alt_names,  # However, the variable name in the file is different to the one Raven expects: use alt-names.
            data_kwds=data_kwds,
        ),
        rc.Gauge.from_nc(
            ERA5_tmin,
            data_type=["TEMP_MIN"],
            alt_names=alt_names,
            data_kwds=data_kwds,
        ),
        rc.Gauge.from_nc(
            ERA5_pr, data_type=["PRECIP"], alt_names=alt_names, data_kwds=data_kwds
        ),
    ],
    # The HRU as defined earlier must be provided. This is the physical representation of the catchment that Raven
    # needs for certain models and processes.
    HRUs=[hru],
    # Model simulation start and end dates.
    StartDate=start_date,
    EndDate=end_date,
    # Name of the simulation. Raven will prefix all .rvX files and model outputs with the runName.
    RunName="test_NB_04",
    # Custom outputs allow pre-processing of certain statistics and variables after the model runs. Please see the
    # documentation for more details on possible options.
    CustomOutput=rc.CustomOutput("YEARLY", "AVERAGE", "PRECIP", "ENTIRE_WATERSHED"),
    # GR4JCN needs an extra constant parameter for the catchment, corresponding to the G50 parameter in the CEMANEIGE
    # description. Here is how we provide it.
    GlobalParameter={"AVG_ANNUAL_RUNOFF": 208.480},
)

# We are now ready to write this newly-configured model to disk through the use of .rvX files that Raven will read.

# In the following code snippet, there is a "workdir" path that must be provided. This indicates the path
# where the data used to run the model (RAVEN .RV files) will be made available from the PAVICS Jupyter environment.
# The default "workdir" setting puts model outputs in the temporary directory ("/tmp"),
# which is not visible from the Jupyter file explorer. Therefore, You can change the last subfolder,
# but '/notebook_dir/writable-workspace/' must be the beginning  of the path when running on the PAVICS platform.

workdir = Path(tempfile.mkdtemp(prefix="NB4"))
m.write_rv(workdir=workdir)

/home/ets/src/RavenPy/ravenpy/utilities/testdata.py:131: UserWarning: notebook_inputs/ERA5_tmax.nc.md5 not accessible online. Unable to determine validity with upstream repo.
  warnings.warn(msg)
/home/ets/src/RavenPy/ravenpy/utilities/testdata.py:131: UserWarning: notebook_inputs/ERA5_tmin.nc.md5 not accessible online. Unable to determine validity with upstream repo.
  warnings.warn(msg)
/home/ets/src/RavenPy/ravenpy/utilities/testdata.py:131: UserWarning: notebook_inputs/ERA5_pr.nc.md5 not accessible online. Unable to determine validity with upstream repo.
  warnings.warn(msg)


{'rvi': PosixPath('/tmp/NB4d1mhun7o/test_NB_04.rvi'),
 'rvp': PosixPath('/tmp/NB4d1mhun7o/test_NB_04.rvp'),
 'rvc': PosixPath('/tmp/NB4d1mhun7o/test_NB_04.rvc'),
 'rvh': PosixPath('/tmp/NB4d1mhun7o/test_NB_04.rvh'),
 'rvt': PosixPath('/tmp/NB4d1mhun7o/test_NB_04.rvt')}

In [6]:
# 2- Alternatively, we could already have (or we could create) a single netcdf file containing all the required
#    forcing data. Since we computed such a merged file in Notebook 03, let's reuse it here

# Example for using the data we just generated in Notebook 03:
"""
ERA5_full = ERA5_weather_data.nc
"""

# In our case, we will prefer to link to existing, pre-computed and locally stored files to keep things tidy:
ERA5_full = get_file("notebook_inputs/ERA5_weather_data.nc")

#    Since our meteorological gauge data is all included in a single file, we need to tell the model which variables
#    we are providing. We will generate the list now and pass it later to Ravenpy as an argument to the model.
data_type = ["TEMP_MAX", "TEMP_MIN", "PRECIP"]

#    Now that we have the single file containing tmax, tmin and pr, we can setup a single gauge that contains all three.
m = GR4JCN(
    params=[0.529, -3.396, 407.29, 1.072, 16.9, 0.947],
    # Setup the gauge using the second method, i.e., using a single file that contains all meteorological inputs. As
    # you can see, a single gauge is added, but it contains all the information we need.
    Gauge=[
        rc.Gauge.from_nc(
            ERA5_full,  # Path to the ERA5 file containing all three meteorological variables
            data_type=data_type,  # Note that this is the list of all the variables
            alt_names=alt_names,  # Note that all variables here are mapped to their names in the netcdf file.
            data_kwds=data_kwds,
        )
    ],
    HRUs=[hru],
    StartDate=start_date,
    EndDate=end_date,
    RunName="test_NB04_b",
    CustomOutput=rc.CustomOutput("YEARLY", "AVERAGE", "PRECIP", "ENTIRE_WATERSHED"),
    GlobalParameter={"AVG_ANNUAL_RUNOFF": 208.480},
)

# Now we will write the files to disk to prepare them for Raven. This step is not strictly necessary since the
# next step will also write files to disk automatically. We will leave it here so we can see the intermediate step
# and inspect the files if necessary.

m.write_rv(workdir=workdir)

/home/ets/src/RavenPy/ravenpy/utilities/testdata.py:131: UserWarning: notebook_inputs/ERA5_weather_data.nc.md5 not accessible online. Unable to determine validity with upstream repo.
  warnings.warn(msg)


{'rvi': PosixPath('/tmp/NB4d1mhun7o/test_NB04_b.rvi'),
 'rvp': PosixPath('/tmp/NB4d1mhun7o/test_NB04_b.rvp'),
 'rvc': PosixPath('/tmp/NB4d1mhun7o/test_NB04_b.rvc'),
 'rvh': PosixPath('/tmp/NB4d1mhun7o/test_NB04_b.rvh'),
 'rvt': PosixPath('/tmp/NB4d1mhun7o/test_NB04_b.rvt')}

It can be seen that both methods generated .rvX files in the indicated path. This makes the Ravenpy platform more flexible for various use-cases, where some data can be stored in independent files or databases (perhaps temperatures come from one source and precipitation from another source).

The above code only created the .rvX files. The model did not actually run yet. To do so, we must ask it to, as follows. Don't worry about the warnings: Raven is informing us that it has generated some internal parameters to build the model configuration based on some of the parameters we provided.

In [7]:
# If we want to import our own raven configuration files and forcing data, we can do so by importing them
# using the ravenpy.run  method. This will run the model exactly as the users will have designed it.
from ravenpy import OutputReader
from ravenpy.ravenpy import run

# This is used to specify the raven configuration files prefixes. In this case, files are named test_2.rvi, etc.
run_name = "test_2"

# This is the path where the files were uploaded by the user. Model outputs will also be placed there in a
# subfolder called "outputs"
configdir = workdir

# Run the model and get the path to the outputs folder that can be used in the output reader.
outputs_path = run(modelname=run_name, configdir=configdir)

# Get the outputs using the Output Reader object.
outputs = OutputReader(run_name=run_name, path=outputs_path)

RavenError: Config directory: /tmp/NB4d1mhun7o
Cannot find or read .rvi file

In [ ]:
# If we already have a model configuration that we built in-memory (such as the "m" GR4JCN model we built above),
# then we can use the Emulator object to simply emulate the model we were working on and get outputs directly
from ravenpy import Emulator

# Prepare the emulator by writing files on disk
e = Emulator(config=m)

# Run the model and get the outputs.
outputs = e.run()

However, the above demonstration shows how to use one of the eight emulators to run a Raven simulation. Ravenpy also contains other powerful tools to run other user-defined raven models by reading existing configuration files and running the model in Ravenpy. This means that users that already have Raven models of their systems can upload the configuration and hydrometeorological data netcdf files to their private account on the PAVICS-Hydro server and run their model there. Here is an example of how this could be done.

At this stage, no matter the method we used, we have the outputs of the model simulations in the "outputs" object. Let's explore it:

In [ ]:
# Show the files available in the outputs. Each of these can be accessed to get information about the simulation.
outputs.files

The outputs are as follow:

- hydrograph: The actual simulated hydrograph (q_sim), in netcdf format. It also contains the observed discharge (q_obs) if observed streamflow was provided as a forcing file.
- storage: The state variables of the simulation duration, in netcdf format
- solution: The state variables at the end of the simulation, which are saved as a ".rvc" file that can be used to hot-start a model (for forecasting, for example)
- messages: A list of messages returned by Raven when executing the run.

You can explore the outputs using othe following syntax. This loads the data into memory to be used directly in another cell for processing or analysis.


In [ ]:
# The model outputs are actually already loaded as Python objects in memory, thus we can access the data directly.
print("----------------HYDROGRAPH----------------")
display(outputs.hydrograph)
print("")
print("-----------------STORAGE------------------")
display(outputs.storage)
print("")
print("-----------------SOLUTION-----------------")
display(outputs.solution)
print("")

We can see in the "hydrograph" section that the model has generated a simulation using the forcing data we provided, but it only used the period between the start_date and end_date we asked it for. We can see that the dates of the ERA5 data we requested in the previous notebook cover the period 1980-01-01 to 1991-01-01. In our simulation, we only ask to run over the period from 1985-01-01 to 1990-01-01. Raven takes care of subsetting the data for the required period. We can look at the simulated streamflow from Raven to confirm this:

In [ ]:
# Import the graphing utility built to handle Raven model outputs
from ravenpy.utilities.nb_graphs import hydrographs

hydrograph_objects = outputs.hydrograph
hydrographs(hydrograph_objects)

As you can see, the simulated flow covers only the period we asked for. The results probably don't look good, but that's OK! We will soon calibrate our model to get reasonable parameters.

We could also simply do basic plots using:

In [ ]:
outputs.hydrograph.q_sim.plot()

Finally, we can inspect and work with other state variables in the model outputs. For example, say we want to investigate the snow water equivalent timeseries. We can first get the list of available state variables:

In [ ]:
print(list(outputs.storage.keys()))

And then plot the variable of interest:


In [ ]:
# Plot the "Snow" variable
outputs.storage["Snow"].plot()

As you can see, PAVICS-Hydro makes it easy to build a hydrological model, run it with forcing data, and then interact with the results! In the next notebooks, we will see how to adjust configuration files (the .rvX files) to setup and run a model, and also how to calibrate its parameters.



## Supplementary information on Hydrological response unit definition
Raven requires a description of the watershed streamflow is simulated in. Different models require different parameters, but minimally, area, elevation, latitude and longitude are required. These data need to be provided for a few reasons:
* Area is required since the size of the watershed will directly influence the simulated streamflow. Units are in square kilometers (km²).
* Elevation (average elevation of the watershed) is required, although in many models the value is not actually used and therefore can be set to an arbitrary number. We strongly recommend using the real elevation as that will ensure that the value is present if you decide to switch to another model that requires elevation. Elevation is expressed in meters above mean sea level.
* Latitude and longitude refer to the catchment centroid, and are used, among others, for evapotranspiration  computations. They are expressed in decimal degrees (°), with longitudes within [-180, 180].

These values should be either precomputed externally, or they can be computed using the PAVICS-Hydro geophysical extraction toolbox that we used in the second tutorial notebook.

## Supplementary information on model parameters

Each model requires a set of tuning parameters to represent and compensate for unknown quantities in certain hydrological processes. Some models have more parameters than others, for example:

* GR4JCN = 6 parameters
* HMETS = 21 parameters
* MOHYSE = 10 parameters
* HBVEC = 21 parameters

These parameters are found through calibration by tuning their values until the simulated streamflow matches the observations as much as possible. PAVICS-Hydro provides an integrated calibration toolbox that will be explored in the the 6th step of this tutorial. For now, we simply provided a set of parameters but it is not yet fully calibrated. This explains the poor quality of the simulated hydrograph.

## Explore!
With this information in mind, you can now explore running different models and parameters and on different periods, and display the simulated hydrographs. You can change the start and end dates, the area, latitude, and even add other options that you might find in the documentation or in later tutorials.

If you want to run other models than GR4JCN, you can use these parameter sets:

#### HMETS:
params = (9.5019, 0.2774, 6.3942, 0.6884, 1.2875, 5.4134, 2.3641, 0.0973, 0.0464, 0.1998, 0.0222, -1.0919, 2.6851, 0.3740,
          1.0000, 0.4739, 0.0114, 0.0243, 0.0069, 310.7211, 916.1947)

#### MOHYSE:
params = (1.0, 0.0468, 4.2952, 2.658, 0.4038, 0.0621, 0.0273, 0.0453, 0.9039, 5.6167)

#### HBVEC:
params = (0.059845, 4.07223, 2.00157, 0.034737, 0.09985, 0.506, 3.4385, 38.32455, 0.46066, 0.06304, 2.2778, 4.8737,
          0.5718813, 0.04505643, 0.877607, 18.94145, 2.036937, 0.4452843, 0.6771759, 1.141608, 1.024278)
